In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np
import schedule
import datetime
import os
import time
from datetime import datetime, timedelta
# import time
import os
from bs4 import BeautifulSoup
import requests
from xml.etree.ElementTree import XML, fromstring, tostring
import pandas as pd
# %matplotlib inline
from matplotlib import pyplot as plt

path = 'D:\Anuj\Here\Ludhiana_New25_6_23'   

In [2]:
page = requests.get("https://traffic.ls.hereapi.com/traffic/6.2/flow.xml?apiKey=BofRfA3YMcL6BLZnsnQQlQJ-87EJbpKwt3HnxWYnBLs&bbox=30.9146,75.9632;30.8795,75.8640&responseattributes=sh,fc")
soup = BeautifulSoup(page.text, "lxml")
roads = soup.find_all('fi')
#print(soup)
roads

c:\ProgramData\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


[<fi><tmc de="Brown Road" le="0.02126" pc="6700" qd="+"></tmc><shp fc="3">30.90879,75.86399 30.90871,75.86401 30.9086,75.86401</shp><cf cn="0.7" ff="24.0" jf="3.88888" sp="17.0" su="17.0" ty="TR"></cf></fi>,
 <fi><tmc de="Janakpuri Road" le="0.84554" pc="6699" qd="+"></tmc><shp fc="3">30.9086,75.86401 30.90848,75.864 30.90782,75.86386</shp><shp fc="3">30.90782,75.86386 30.90761,75.86383</shp><shp fc="3">30.90761,75.86383 30.90742,75.8638</shp><shp fc="3">30.90742,75.8638 30.90728,75.86378 30.90717,75.86377</shp><shp fc="3">30.90717,75.86377 30.90706,75.86377 30.90692,75.86379 30.90685,75.86381</shp><shp fc="3">30.90685,75.86381 30.90674,75.86386 30.90661,75.86394</shp><shp fc="3">30.90661,75.86394 30.90653,75.86398</shp><shp fc="3">30.90653,75.86398 30.90634,75.8641 30.90624,75.86415</shp><shp fc="3">30.90624,75.86415 30.90596,75.86425</shp><shp fc="3">30.90596,75.86425 30.90588,75.86428</shp><shp fc="3">30.90588,75.86428 30.90558,75.86447</shp><shp fc="3">30.90558,75.86447 30.90531,75

In [36]:
def get_traffic_data():
    data_dict = {"names":[],"speed":[],"lattitude":[],"Longitude":[],"fc":[],'le':[],'qd':[],'cn':[], 'ff':[], 'jf':[], 'sp':[],'ty':[]}

    #print("LOOP")
    for road in roads:
        myxml = fromstring(str(road))
        fc=5
        for child in myxml:
    #         print(child.tag, child.arrrib)
            if 'de' in child.attrib:
                de=str(child.attrib['de'])
            if 'qd' in child.attrib:
                qd=str(child.attrib['qd'])
            if 'le' in child.attrib:
                le=float(child.attrib['le'])
            if 'fc' in child.attrib:
                fc=int(child.attrib['fc'])
            if 'cn' in child.attrib:
                cn=float(child.attrib['cn'])
            if 'su' in child.attrib:
                su=float(child.attrib['su'])
            if 'ff' in child.attrib:
                ff=float(child.attrib['ff'])
            if 'jf' in child.attrib:
                jf=float(child.attrib['jf'])
            if 'sp' in child.attrib:
                sp=float(child.attrib['sp'])
            if 'ty' in child.attrib:
                ty=child.attrib['ty']
        #if fc <= 4 and cn >=0.7:
        shps = road.find_all('shp')
        #print("IN LOOP ")
        #print(len(shps))
        for j in range(len(shps)): # going through all the shapes in a FI
            coords = shps[j].text.replace(',',' ').split() #list
            las = []
            lons = []
            for i in range(int(len(coords)/2)): # going through all the lat longs in a shp
                las.append(float(coords[2*i]))
                lons.append(float(coords[2*i+1]))
            #loc_pros.append([float(su),float(ff)])
            #data_dict['Road_Name'].append([de])
            data_dict['lattitude'].append(las)
            data_dict['Longitude'].append(lons)
            #lats.append(las)
            #longs.append(lons)
            data_dict['names'].append(de)
            data_dict['speed'].append(float(su))
            data_dict['fc'].append(fc)
            data_dict['le'].append(le)
            data_dict['qd'].append(qd)
            data_dict['cn'].append(cn)
            data_dict['ff'].append(ff)
            data_dict['jf'].append(jf)
            data_dict['sp'].append(sp)
            data_dict['ty'].append(ty)

    df=pd.DataFrame(data_dict)
    return df

In [34]:
df

,names,speed,lattitude,Longitude,fc,le,qd,cn,ff,jf,sp,ty
0,Brown Road,17.00,"[30.90879, 30.90871, 30.9086]","[75.86399, 75.86401, 75.86401]",3,0.02126,+,0.70,24.0,3.88888,17.00,TR
1,Janakpuri Road,18.31,"[30.9086, 30.90848, 30.90782]","[75.86401, 75.864, 75.86386]",3,0.84554,+,0.74,25.0,3.56698,18.31,TR
2,Janakpuri Road,18.31,"[30.90782, 30.90761]","[75.86386, 75.86383]",3,0.84554,+,0.74,25.0,3.56698,18.31,TR
3,Janakpuri Road,18.31,"[30.90761, 30.90742]","[75.86383, 75.8638]",3,0.84554,+,0.74,25.0,3.56698,18.31,TR
4,Janakpuri Road,18.31,"[30.90742, 30.90728, 30.90717]","[75.8638, 75.86378, 75.86377]",3,0.84554,+,0.74,25.0,3.56698,18.31,TR
...,...,...,...,...,...,...,...,...,...,...,...,...
5680,Bhamian Jhabewal Main Road/Mundian Road,22.00,"[30.89896, 30.89894]","[75.95326, 75.95368]",4,0.49446,-,0.70,28.0,2.85714,22.00,TR
5681,Bhamian Jhabewal Main Road/Mundian Road,22.00,"[30.89894, 30.89893]","[75.95368, 75.95399]",4,0.49446,-,0.70,28.0,2.85714,22.00,TR
5682,Bhamian Jhabewal Main Road/Mundian Road,22.00,"[30.89893, 30.89891]","[75.95399, 75.95453]",4,0.49446,-,0.70,28.0,2.85714,22.00,TR
5683,Bhamian Jhabewal Main Road/Mundian Road,22.00,"[30.89891, 30.8989]","[75.95453, 75.95487]",4,0.49446,-,0.70,28.0,2.85714,22.00,TR


In [35]:
df.to_csv("example4.csv")

In [ ]:
def getTime():
    now = datetime.datetime.now()
    current_time = now.strftime("%d-%b-%Y-%H-%M")
    return current_time

def realTime_to_schedule():    
    outFile = path + '\\' + getTime() + '.csv'
    data = get_traffic_data()

    if data is None:
        pass
    else:
        print("Writing data to " + outFile)
        data.to_csv(outFile)
        
schedule.every(5).minutes.do(realTime_to_schedule)
# warnings.filterwarnings("ignore")

while True:
    schedule.run_pending()
    time.sleep(1)